In [ ]:
#mounting drive to colab if not done already
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
#Adding required libraries
!pip install --upgrade pytube
!pip install opencv-python
!pip install youtube_transcript_api
!pip install psycopg2-binary
!pip install --upgrade tensorflow
!pip install torch torchvision
!pip install tensorflow-hub

  Using cached tensorflow-2.17.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (601.3 MB)
  Using cached tensorboard-2.17.0-py3-none-any.whl (5.5 MB)
  Using cached keras-3.4.1-py3-none-any.whl (1.1 MB)
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.2
    Uninstalling tensorboard-2.15.2:
      Successfully uninstalled tensorboard-2.15.2
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.15.1
    Uninstalling tensorflow-2.15.1:
      Successfully uninstalled tensorflow-2.15.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.15.1 requires tensorflow<2.16,>=2.15, but you have tensorflow 2.17.0 which is incompatible.
  Using c

In [ ]:
#Imports
import cv2
import os
import csv
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.transforms import functional as F
import pandas as pd
import torch

def frameExtract(vPath, outFolder):
    os.makedirs(outFolder, exist_ok=True)
    videoCapture = cv2.VideoCapture(vPath)
    frameCount = int(videoCapture.get(cv2.CAP_PROP_FRAME_COUNT))

    for frameNum in range(frameCount):
        ret, frame = videoCapture.read()
        if not ret:
            break
        #Save frames in given folder below
        fName = os.path.join(outFolder, f"frame{frameNum:04d}.jpg")
        cv2.imwrite(fName, frame)

    videoCapture.release()

#Path where we store youtube video
vPath = input("Enter path where youtube video exist: ")
#Path where we want to store the frames
outFolder = input("Enter folder path to store the frames: ")
frameExtract(vPath, outFolder)

In [ ]:
#Training and exec phase
model = fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()

#load COCO class for obj detection
#Note: cocoPath is a place holder that should be changed later with path for cocoFile
with open(cocoPath, "r") as f:
    cocoLabel = f.read().strip().split("\n")

#A local directory to pull the frames
frameDir = fDirLocation
columns = ["vidId", "frameNum", "timestamp", "detectedObjId", "detectedObjClass", "confidence", "bbox info"]
detectArr = []

for vidId, fFile in enumerate(os.listdir(frameDir)):
    frame_path = os.path.join(frameDir, fFile)
    frame = cv2.imread(frame_path)
    frame_tensor = F.to_tensor(frame).unsqueeze(0)
    with torch.no_grad():
        detections = model(frame_tensor)

    for detection in detections[0]["boxes"]:
        boxes, labels, scores = detection, detections[0]["labels"], detections[0]["scores"]
        for box, label, score in zip(boxes, labels, scores):
            if score > 0.7:
                bbox_info = box.tolist()

                detectArr.append({
                    "vidId": vidId,
                    "frameNum": fFile,
                    "timestamp": None,    #Currently set to none, but later will be taken from caption file
                    "detectedObjId": label.item(),
                    "detectedObjClass": cocoLabel[label.item()],
                    "confidence": score.item(),
                    "bbox info": bbox_info
                })

detectData = pd.DataFrame(detectArr, columns=columns)

#Note: outCSV is a placeholder for path where file needs to be stored
detectData.to_csv(output_csv, index=False)